In [246]:
#fantasy football python schedule maker
# do a double round robin, take the first set of pairings and then add in w-n pairings where w is weeks and n is number of teams

import random
import pandas as pd
from IPython.display import clear_output

n_weeks = 14
teams = ['Charlie','Sam','Ben','Dede','Nicole','Cyril','Joe','Alexandra','Mike','Marvin','Zach','Matt']


In [249]:

#initialize configurations of the round robin

def round_robin_teams(teams, weeks=None):
    random.shuffle(teams)
    s = []
    if len(teams) % 2 == 1: teams = teams + [None]

    n_teams = len(teams)
    index = list(range(n_teams))
    midpoint = n_teams//2

    #handle a way to get a schedule for weeks < teams
    if weeks:
        weeks = min(n_teams-1,weeks)
    else:
        weeks = n_teams-1

    for i in range(weeks):
        list_1 = index[:midpoint]
        list_2 = index[midpoint:]
        list_2.reverse()
        round = []
        for j in range(midpoint):
            team_1 = teams[list_1[j]]
            team_2 = teams[list_2[j]]
            if j==0 and i%2 == 1:
                round.append(sorted((team_2,team_1)))
            else:
                round.append(sorted((team_1,team_2)))
        s.append(round)
        index = index[midpoint:-1] + index[:midpoint] + index[-1:]

    return s

def ff_scheduler(weeks,teams):
    if weeks > len(teams)-1:
        remaining_weeks = weeks - (len(teams)-1)
        schedule = list(round_robin_teams(teams)) + list(round_robin_teams(teams,weeks=remaining_weeks))
    else:
        schedule = list(round_robin_teams(teams))
    return schedule


def melt_schedule(ff_sc:list):
    test = pd.DataFrame(ff_sc)
    test['week'] = [i + 1 for i in list(test.index)]
    tmp = pd.DataFrame()
    for i in [n for n in test.columns if n!='week']:
        z = test[[i,'week']]
        z.columns = ['mu','week']
        tmp =pd.concat([tmp,z])
    tmp['mu'] = tmp['mu'].astype(str)
    tmp['mu'] = tmp['mu'].apply(lambda x: x.replace("'",""))
    return tmp


def print_schedule(ff_sc:list):
    return [print(i) for i in ff_sc]



def TEST_rapid(schedule:list,rapid_rate:int,return_dupes=False):
    tmp = melt_schedule(schedule)
    tmp2 = tmp.merge(tmp,on=['mu'])
    tmp3 = tmp2[(tmp2['week_x']!=tmp2['week_y'])&(tmp2['week_x']<tmp2['week_y'])]
    rematches = tmp3.copy()
    rematches['week_diff'] = rematches['week_y'] - rematches['week_x']
    if return_dupes:
        return rematches
    if rematches[rematches['week_diff']<=rapid_rate].shape[0] > 0: #fails the test
        #print('FAILED: rapid')
        return False
    else:
        return True
        
def TEST_marquee(schedule:list,marquee:list):
    #make sure that the marquee matchups are happening in thier own distinct week
    df = melt_schedule(schedule)
    mask = df['mu'].isin(marquee)
    if (df[mask].groupby('mu').min().reset_index())['week'].nunique()>= len(marquee): #passes the test
        if (df[mask].groupby('mu').min().reset_index())['week'].isin([1,14]):
            return False
        else:
            return True
    else:
        #print('FAILED: marquee')
        return False

def TEST_homebuilder(schedule,matchup,week):
    return True
    # df = melt_schedule(schedule)
    # if df[(df['week']==week) & (df['mu'] == matchup)].shape[0]>0: # passes the test
    #     return True
    # else:
    #     #print('FAILED: homebuilder')
    #     return False

In [260]:
#homebuilder
week = 4
matchup = '[Charlie, Nicole]'

#marquee
marquee = ['[Charlie, Sam]', #Lederer
           '[Charlie, Nicole]', #Homemaker
           '[Alexandra, Joe]', #Homewrecker
           '[Marvin, Sam]'] #Rematch

iteration=0
break_flag = False
not_found = True

rapid_fail = 0
homebuilder_fail = 0
marquee_fail = 0

while not_found:

    ff_schedule = ff_scheduler(14,teams)
    shuffle_count = 0
    while shuffle_count<30:
        random.shuffle(ff_schedule)
        if TEST_marquee(ff_schedule,marquee):
            if TEST_homebuilder(ff_schedule,matchup,week):
                if TEST_rapid(ff_schedule,3):
                    print('SCHEDULE FOUND')
                    not_found = False
                    break_flag = True
                    shuffle_count = 30
                    break 
                else:
                    rapid_fail +=1
                    shuffle_count+=1
            else:
                if break_flag:
                    break
                homebuilder_fail+=1
                shuffle_count+=1
        else:
            if break_flag:
                break
            marquee_fail+=1
            shuffle_count = 30

        

    clear_output(wait=True)
    print(f'''MARQUEE FAILS: {marquee_fail}
HOMEBUILDER FAILS: {homebuilder_fail}
RAPID FAIL: {rapid_fail}''')

        
        

MARQUEE FAILS: 71
HOMEBUILDER FAILS: 1937
RAPID FAIL: 187
[['Cyril', 'Joe'], ['Marvin', 'Matt'], ['Charlie', 'Dede'], ['Alexandra', 'Sam'], ['Nicole', 'Zach'], ['Ben', 'Mike']]
[['Alexandra', 'Ben'], ['Mike', 'Sam'], ['Joe', 'Nicole'], ['Matt', 'Zach'], ['Charlie', 'Cyril'], ['Dede', 'Marvin']]
[['Cyril', 'Mike'], ['Ben', 'Nicole'], ['Sam', 'Zach'], ['Alexandra', 'Dede'], ['Charlie', 'Marvin'], ['Joe', 'Matt']]
[['Cyril', 'Marvin'], ['Dede', 'Joe'], ['Matt', 'Sam'], ['Charlie', 'Nicole'], ['Alexandra', 'Mike'], ['Ben', 'Zach']]
[['Cyril', 'Matt'], ['Charlie', 'Joe'], ['Alexandra', 'Marvin'], ['Dede', 'Zach'], ['Ben', 'Sam'], ['Mike', 'Nicole']]
[['Cyril', 'Sam'], ['Dede', 'Nicole'], ['Marvin', 'Mike'], ['Ben', 'Joe'], ['Matt', 'Zach'], ['Alexandra', 'Charlie']]
[['Alexandra', 'Cyril'], ['Charlie', 'Zach'], ['Ben', 'Matt'], ['Joe', 'Mike'], ['Marvin', 'Nicole'], ['Dede', 'Sam']]
[['Charlie', 'Cyril'], ['Alexandra', 'Matt'], ['Joe', 'Zach'], ['Ben', 'Marvin'], ['Dede', 'Mike'], ['Nicole'

In [261]:
pd.DataFrame(ff_schedule).to_clipboard()

In [259]:
print(f'''MARQUEE FAILS: {marquee_fail}
HOMEBUILDER FAILS: {homebuilder_fail}
RAPID FAIL: {rapid_fail}''')

MARQUEE FAILS: 56
HOMEBUILDER FAILS: 1550
RAPID FAIL: 166


In [262]:
z = TEST_rapid(ff_schedule,rapid_rate = 3,return_dupes=True)
z

,mu,week_x,week_y,week_diff
1,"[Cyril, Joe]",1,13,12
5,"[Alexandra, Ben]",2,9,7
11,"[Cyril, Matt]",5,12,7
18,"[Charlie, Cyril]",2,8,6
25,"[Alexandra, Dede]",3,12,9
29,"[Alexandra, Mike]",4,13,9
33,"[Marvin, Matt]",1,13,12
37,"[Mike, Sam]",2,10,8
41,"[Ben, Nicole]",3,13,10
52,"[Charlie, Marvin]",3,12,9


In [265]:
df = melt_schedule(ff_schedule)
mask = df['mu'].isin(marquee)
(df[mask].groupby('mu').min().reset_index())['week'].isin([4])


0    False
1     True
2    False
3    False
Name: week, dtype: bool

In [239]:
TEST_marquee(ff_schedule,marquee)

FAILED: marquee


False